In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [10]:
df = pd.read_csv('/content/new_car_data.csv')

df.drop(columns=['Unnamed: 0', 'New_Price'], inplace=True)
df = df[df.Owner_Type != "Fourth & Above"]
df = df[(df.Fuel_Type != "Electric") & (df.Fuel_Type != "LPG")]

In [11]:
df.replace({'Fuel_Type':{'Petrol':0, 'Diesel':1, 'CNG':2},
            'Transmission':{'Manual':0, 'Automatic':1},
           'Owner_Type':{'First':0, 'Second':1, 'Third':2}}, inplace=True)

<ipython-input-11-a4ae993482ea>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Fuel_Type':{'Petrol':0, 'Diesel':1, 'CNG':2},


In [12]:
df = pd.get_dummies(df, columns=['Name', 'Location'], drop_first=True, dtype=int)

In [13]:
df.head()

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,...,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,Location_Pune
0,2010,72000,2,0,0,26.60,998.0,58.16,5.0,1.75,...,0,0,0,0,0,0,0,0,1,0
1,2015,41000,1,0,0,19.67,1582.0,126.20,5.0,12.50,...,0,0,0,0,0,0,0,0,0,1
2,2011,46000,0,0,0,18.20,1199.0,88.70,5.0,4.50,...,0,1,0,0,0,0,0,0,0,0
3,2012,87000,1,0,0,20.77,1248.0,88.76,7.0,6.00,...,0,1,0,0,0,0,0,0,0,0
4,2013,40670,1,1,1,15.20,1968.0,140.80,5.0,17.74,...,0,0,1,0,0,0,0,0,0,0


In [14]:
df.columns

Index(['Year', 'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Owner_Type',
       'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Name_Audi', 'Name_BMW',
       'Name_Bentley', 'Name_Chevrolet', 'Name_Datsun', 'Name_Fiat',
       'Name_Force', 'Name_Ford', 'Name_Honda', 'Name_Hyundai', 'Name_ISUZU',
       'Name_Isuzu', 'Name_Jaguar', 'Name_Jeep', 'Name_Lamborghini',
       'Name_Land', 'Name_Mahindra', 'Name_Maruti', 'Name_Mercedes-Benz',
       'Name_Mini', 'Name_Mitsubishi', 'Name_Nissan', 'Name_Porsche',
       'Name_Renault', 'Name_Skoda', 'Name_Smart', 'Name_Tata', 'Name_Toyota',
       'Name_Volkswagen', 'Name_Volvo', 'Location_Bangalore',
       'Location_Chennai', 'Location_Coimbatore', 'Location_Delhi',
       'Location_Hyderabad', 'Location_Jaipur', 'Location_Kochi',
       'Location_Kolkata', 'Location_Mumbai', 'Location_Pune'],
      dtype='object')

In [23]:
 df.dropna(inplace = True)

In [32]:
df.Price.describe()

,Price
count,5855.000000
mean,9.623291
std,11.259869
min,0.440000
25%,3.550000
50%,5.750000
75%,10.085000
max,160.000000


#### Data Splitting and Preprocessing

In [25]:
X = df.drop('Price', axis=1)
y = df['Price']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
X_train[0]

array([ 1.40526552, -0.14947853,  0.86962263, -0.63633247, -0.44202318,
        1.21720782, -0.43029004, -0.85229655, -0.35081471, -0.2033512 ,
       -0.20953428, -0.01461295, -0.14154449, -0.0506803 , -0.06211034,
       -0.02531578, -0.23644497, -0.33868464, -0.47017402, -0.01461295,
       -0.01461295, -0.08162333, -0.0566805 , -0.01461295, -0.09625614,
       -0.21556916, -0.50106724, -0.24044524, -0.07024645, -0.06381914,
       -0.12227529, -0.0547527 , -0.1655847 , -0.1709513 ,  0.        ,
       -0.17873266,  3.78911296, -0.23644497, -0.06381914, -0.2468373 ,
       -0.30094922,  2.87348517, -0.3183398 , -0.37333918, -0.27124386,
       -0.3476238 , -0.30972386, -0.39348742, -0.33868464])

#### Building Our Neural Network

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])


model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 118.9221 - root_mean_squared_error: 10.7184 - val_loss: 23.2853 - val_root_mean_squared_error: 4.8255
Epoch 2/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 23.3365 - root_mean_squared_error: 4.8194 - val_loss: 17.1422 - val_root_mean_squared_error: 4.1403
Epoch 3/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.4468 - root_mean_squared_error: 3.7918 - val_loss: 14.5423 - val_root_mean_squared_error: 3.8134
Epoch 4/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14.1160 - root_mean_squared_error: 3.7309 - val_loss: 13.6799 - val_root_mean_squared_error: 3.6986
Epoch 5/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12.6007 - root_mean_squared_error: 3.5333 - val_loss: 12.2062 - val_root_mean_squared_error: 3.4937
Epoch 6/30
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12.9173 - root_mean_squared_error: 3.5846 - val_loss: 12.9080 - val_root_mean_squared_error: 3.5928
Epoch 7/30
147/147 ━━━━━━━━━━━━━━━━━━━

In [30]:
loss, rmse = model.evaluate(X_test, y_test)

print("Root Mean Squared Error (RMSE) of the model is:", rmse)
print("Loss (MSE) of the model is:", loss)

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.4463 - root_mean_squared_error: 3.1707
Root Mean Squared Error (RMSE) of the model is: 3.727769136428833
Loss (MSE) of the model is: 13.896262168884277


#### Predicting Data

In [35]:
car_names = ['Name_Audi', 'Name_BMW',
       'Name_Bentley', 'Name_Chevrolet', 'Name_Datsun', 'Name_Fiat',
       'Name_Force', 'Name_Ford', 'Name_Honda', 'Name_Hyundai', 'Name_ISUZU',
       'Name_Isuzu', 'Name_Jaguar', 'Name_Jeep', 'Name_Lamborghini',
       'Name_Land', 'Name_Mahindra', 'Name_Maruti', 'Name_Mercedes-Benz',
       'Name_Mini', 'Name_Mitsubishi', 'Name_Nissan', 'Name_Porsche',
       'Name_Renault', 'Name_Skoda', 'Name_Smart', 'Name_Tata', 'Name_Toyota',
       'Name_Volkswagen', 'Name_Volvo']

loc_names = ['Location_Bangalore',
       'Location_Chennai', 'Location_Coimbatore', 'Location_Delhi',
       'Location_Hyderabad', 'Location_Jaipur', 'Location_Kochi',
       'Location_Kolkata', 'Location_Mumbai', 'Location_Pune']

In [37]:
car_name = input("Enter Your Car Name : ")
loc_name = input("Enter Your Location Name : ")

Enter Your Car Name : Maruti
Enter Your Location Name : Mumbai


In [38]:
car_name_dummies = pd.DataFrame([[1 if car == car_name else 0 for car in car_names]], columns=car_names)

car_loc_dummies = pd.DataFrame([[1 if loc == loc_name else 0 for loc in loc_names]], columns=loc_names)

In [43]:
single_car_features = {
    'Year': 2016,
    'Kilometers_Driven':7000,
    'Fuel_Type': 2,
    'Transmission': 1,
    'Owner_Type': 1,
    'Mileage': 26.6,
    'Engine': 998,
    'Power': 58.16,
    'Seats': 5.0,

}

single_car_df = pd.DataFrame([single_car_features])

single_car_full_df = pd.concat([single_car_df, car_name_dummies, car_loc_dummies], axis=1)

single_car_full_df.head()

,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Name_Audi,...,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,Location_Pune
0,2016,7000,2,1,1,26.6,998,58.16,5.0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
single_car_scaled = scaler.transform(single_car_full_df)

In [45]:
predicted_price = model.predict(single_car_scaled)

print(f"Predicted price of the car is: {predicted_price[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted price of the car is: 3.5728015899658203
